In [4]:
#!/usr/bin/env python3

import time
import random
import gym
import gym_minigrid
import numpy as np
%matplotlib auto

# Load the gym environment
env = gym.make('MiniGrid-Empty-8x8-v0')
env.reset()

gridDimension = 6

qvalues = np.zeros((gridDimension, gridDimension, 4, 3))
#4, 7, 7, 2, 4, 3
#2 (wall or no wall + gray or green)
#4 is direction
#3 is actions
#7, 7, 2 is grid info
#4 is information from 4 observations

discount = 0.9
gamma = 0.8

sum = 0
for i in range(0, 100):
    
    rew = 0
    steps = 0
    x = 0
    y = 0
    direction = 1 #North = 0, East = 1, South = 2, West = 3
    episode = []
    
    for j in range(0, 100):
        #print("step {}".format(i))
    
        # Pick a random action
        action = random.randint(0, env.action_space.n - 5)

        obs, reward, done, info = env.step(action)
        episode.append([x, y, direction, action])
        
        if action == 0:
            direction -= 1
            if direction == -1:
                direction = 3
        elif action == 1:
            direction += 1
            if direction == 4:
                direction = 0
        else:
            if direction == 0 and y < gridDimension - 1:
                y += 1
            elif direction == 1 and x < gridDimension - 1:
                x += 1
            elif direction == 2 and y > 0:
                y -= 1
            elif direction == 3 and x > 0:
                x -= 1
        
        steps += 1

        #env.render()

        #time.sleep(0.01)
        
        if done:
            rew = reward
            break
    
    env.reset()
    sum += steps
    
    if rew == 0:
        continue;

    exp = len(episode) - 1
    for x in episode:
        current = qvalues[x[0]][x[1]][x[2]][x[3]]
        if current == 0:
            qvalues[x[0]][x[1]][x[2]][x[3]] = rew * discount ** exp
        else:
            qvalues[x[0]][x[1]][x[2]][x[3]] += rew * discount ** exp
        exp -= 1
        
print("Randomized model average steps (including ones that failed at 100 steps):")
print(1.0 * sum / 100)
print(qvalues)

Using matplotlib backend: MacOSX
Randomized model average steps (including ones that failed at 100 steps):
96.03
[[[[8.45907663e-02 2.88462874e-01 6.45981371e-02]
   [2.98348824e-01 2.09762941e-01 1.24319912e+00]
   [1.26671971e+00 2.41794273e-01 9.63069899e-01]
   [3.90853256e-01 9.55377762e-02 8.75435063e-02]]

  [[7.14120868e-02 1.29465949e-01 1.52769808e-01]
   [4.54607424e-03 1.73586610e-01 2.03396499e-02]
   [4.91591514e-02 2.73763205e-01 6.60739399e-01]
   [2.19459370e-01 2.49138849e-01 5.96917039e-02]]

  [[7.99038586e-01 2.40772542e-01 5.50338043e-03]
   [1.45563422e-01 8.51920758e-02 3.74949216e-02]
   [6.52835416e-04 9.07956448e-02 4.92249600e-01]
   [4.39259097e-01 6.42449827e-01 7.95975595e-01]]

  [[5.40441641e-03 7.10450732e-04 1.12992922e-02]
   [1.01693629e-02 9.02657368e-03 9.29396535e-02]
   [1.00211580e-01 0.00000000e+00 8.77099669e-04]
   [6.67211902e-03 0.00000000e+00 6.00490712e-03]]

  [[4.00074734e-02 0.00000000e+00 1.25547691e-02]
   [0.00000000e+00 4.75105149

In [5]:
print("Trained model steps:")
env.reset()
env.render()
time.sleep(2)

x = 0
y = 0
direction = 1 #North = 0, East = 1, South = 2, West = 3
steps = 0
for i in range(0, 100):
    maxreward = 0
    actiontotake = 0;
    index = 0
    print([x, y, direction])
    print(qvalues[x][y][direction])
    for value in qvalues[x][y][direction]:
        if value > maxreward:
            maxreward = value
            actiontotake = index
        index += 1
    print(actiontotake)
    
    if actiontotake == 0:
        direction -= 1
        if direction == -1:
            direction = 3
    elif actiontotake == 1:
        direction += 1
        if direction == 4:
            direction = 0
    else:
        if direction == 0 and y < gridDimension - 1:
            y += 1
        elif direction == 1 and x < gridDimension - 1:
            x += 1
        elif direction == 2 and y > 0:
            y -= 1
        elif direction == 3 and x > 0:
            x -= 1

    steps += 1
    
    reward = env.step(actiontotake)[1]
    
    env.render()
    time.sleep(0.1)
    
    if reward > 0:
        break

print(steps)
time.sleep(2)

# Test the close method
env.close()

Trained model steps:
[0, 0, 1]
[0.29834882 0.20976294 1.24319912]
2
[1, 0, 1]
[0.60949551 0.34050849 2.93093169]
2
[2, 0, 1]
[1.16293548 0.16477586 3.04198462]
2
[3, 0, 1]
[0.91660694 0.57562502 3.81845619]
2
[4, 0, 1]
[0.93745473 1.86852398 6.32110542]
2
[5, 0, 1]
[0.52486642 2.36700398 1.44692329]
1
[5, 0, 2]
[1.94954596 1.15644676 4.23926548]
2
[5, 0, 2]
[1.94954596 1.15644676 4.23926548]
2
[5, 0, 2]
[1.94954596 1.15644676 4.23926548]
2
[5, 0, 2]
[1.94954596 1.15644676 4.23926548]
2
[5, 0, 2]
[1.94954596 1.15644676 4.23926548]
2
11
